## CIFAR-100 Retrained vs Redistributed Accuracy Table

### All Seed

In [ ]:
# %%
import os
import numpy as np
import pandas as pd
from lib.utils import model_results_from_npz  # Use your existing function

# %%
# ----- Config -----
NUM_CLASSES = 100
#SEEDS = [42, 602, 311, 637, 800, 543, 969, 122, 336, 93]  # example seeds
#EPOCHS = 50
CLASSES = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 
    'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 
    'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 
    'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 
    'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 
    'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 
    'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 
    'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 
    'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 
    'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 
    'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm'
    ]
#REMOVE_CLASSES = [
    #4, 7, 6, 11, 14, 15, 19, 24, 23, 25, 29, 30, 
    #31, 36, 35, 39, 44, 41, 49, 48, 51, 53, 59, 57, 60, 61, 
    #68, 67, 72, 71, 76, 77, 80, 84, 88, 85, 92, 94, 99, 97] # CIFAR-100

SEEDS = [311, 637, 969]  # example seeds
EPOCHS = 200
REMOVE_CLASSES = [None, 14, 23, 35, 49, 53, 61, 68, 72, 88, 97]

DIR_RETRAIN = "../results/cifar100/"
DIR_REDIS   = "../results/redis_cifar100/"

# %%
# ----- Collect results per removed class and per seed -----
all_results = {removed_idx: [] for removed_idx in REMOVE_CLASSES}

for seed in SEEDS:
    print(f"Processing seed {seed}...")
    for removed_idx in REMOVE_CLASSES:
        removed_name = CLASSES[removed_idx]

        retrain_path = os.path.join(DIR_RETRAIN, f"cifar100_resnet18_s{seed}_e{EPOCHS}_r{removed_idx}.npz")
        redis_path   = os.path.join(DIR_REDIS,   f"cifar_resnet_s{seed}_e{EPOCHS}_rd{removed_idx}.npz")

        if not (os.path.exists(retrain_path) and os.path.exists(redis_path)):
            print(f"  Skipping missing class {removed_name}")
            continue

        # Load models
        model_retrain = model_results_from_npz(retrain_path, NUM_CLASSES)
        model_redis   = model_results_from_npz(redis_path, NUM_CLASSES)

        # Forget and retain accuracy
        forget_retrain = model_retrain.accuracy[removed_idx]
        retain_retrain = np.mean([model_retrain.accuracy[i] for i in range(NUM_CLASSES) if i != removed_idx])

        forget_redis = model_redis.accuracy[removed_idx]
        retain_redis = np.mean([model_redis.accuracy[i] for i in range(NUM_CLASSES) if i != removed_idx])

        all_results[removed_idx].append({
            "forget_retrain": forget_retrain,
            "retain_retrain": retain_retrain,
            "forget_redis": forget_redis,
            "retain_redis": retain_redis,
        })

# %%
# ----- Average results across seeds -----
rows = []
for removed_idx in REMOVE_CLASSES:
    removed_name = CLASSES[removed_idx]

    seed_results = all_results[removed_idx]
    if not seed_results:
        continue

    avg_forget_retrain = np.nanmean([r["forget_retrain"] for r in seed_results])
    avg_retain_retrain = np.mean([r["retain_retrain"] for r in seed_results])
    avg_forget_redis  = np.nanmean([r["forget_redis"]  for r in seed_results])
    avg_retain_redis  = np.mean([r["retain_redis"]  for r in seed_results])

    rows.append({
        "removed_class": removed_idx,
        "removed_name": removed_name,
        "forget_retrain": avg_forget_retrain,
        "retain_retrain": avg_retain_retrain,
        "forget_redis": avg_forget_redis,
        "retain_redis": avg_retain_redis,
    })

# %%
# ----- Build DataFrame -----
df_avg = pd.DataFrame(rows)
print(df_avg)

# Optionally save:
df_avg.to_csv("../analytics/CIFAR100/accuracy/redis_vs_retrain_accuracy_all_seed.csv", index=False)


Processing seed 42...
Processing seed 602...
Processing seed 311...
Processing seed 637...
Processing seed 800...
Processing seed 543...
Processing seed 969...
Processing seed 122...
Processing seed 336...
Processing seed 93...
    removed_class removed_name  forget_retrain  retain_retrain  forget_redis  \
0               4       beaver             0.0        0.736182           0.0   
1               7       beetle             0.0        0.735980           0.0   
2               6          bee             0.0        0.733525           0.0   
3              11          boy             0.0        0.739242           0.0   
4              14    butterfly             0.0        0.736879           0.0   
5              15        camel             0.0        0.736747           0.0   
6              19       cattle             0.0        0.735232           0.0   
7              24    cockroach             0.0        0.733323           0.0   
8              23        cloud             0.0      

### One Seed

In [4]:
# %% [markdown]
# # CIFAR-10 Retrained vs Redistributed Accuracy Table

# %%
import os
import numpy as np
import pandas as pd

from lib.utils import model_results_from_npz  # Use your existing function

# %%
# ----- Config -----
NUM_CLASSES = 100
SEED = 969
EPOCHS = 50
CLASSES = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 
    'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 
    'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 
    'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 
    'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 
    'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 
    'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 
    'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 
    'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 
    'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 
    'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm'
    ]
REMOVE_CLASSES = [
    4, 7, 6, 11, 14, 15, 19, 24, 23, 25, 29, 30, 
    31, 36, 35, 39, 44, 41, 49, 48, 51, 53, 59, 57, 60, 61, 
    68, 67, 72, 71, 76, 77, 80, 84, 88, 85, 92, 94, 99, 97] # CIFAR-100

#REMOVE_CLASSES = [14, 23, 35, 49, 53, 61, 68, 72, 88, 97]

DIR_RETRAIN = "../results/cifar100/"
DIR_REDIS   = "../results/redis_cifar100/"

# %%
# ----- Collect results per removed class -----
rows = []

for removed_idx in REMOVE_CLASSES:
    removed_name = CLASSES[removed_idx]

    retrain_path = os.path.join(DIR_RETRAIN, f"cifar100_resnet18_s{SEED}_e{EPOCHS}_r{removed_idx}.npz")
    redis_path   = os.path.join(DIR_REDIS,   f"cifar_resnet_s{SEED}_e{EPOCHS}_rd{removed_idx}.npz")

    if not (os.path.exists(retrain_path) and os.path.exists(redis_path)):
        print(f"Skipping missing class {removed_name}")
        continue

    # Load models using your existing function
    model_retrain = model_results_from_npz(retrain_path, NUM_CLASSES)
    model_redis   = model_results_from_npz(redis_path, NUM_CLASSES)

    # Forget and retain accuracy
    forget_retrain = model_retrain.accuracy[removed_idx]
    retain_retrain = np.mean([model_retrain.accuracy[i] for i in range(NUM_CLASSES) if i != removed_idx])

    forget_redis = model_redis.accuracy[removed_idx]
    retain_redis = np.mean([model_redis.accuracy[i] for i in range(NUM_CLASSES) if i != removed_idx])

    rows.append({
        "removed_class": removed_idx,
        "removed_name": removed_name,
        "forget_retrain": forget_retrain,
        "retain_retrain": retain_retrain,
        "forget_redis": forget_redis,
        "retain_redis": retain_redis,
    })

# %%
# ----- Build DataFrame -----
df = pd.DataFrame(rows)
df
# Optionally save as CSV:
df.to_csv(f"../analytics/CIFAR100/accuracy/redis_vs_retrain_accuracy_e{EPOCHS}_s{SEED}.csv", index=False)


# With Pretrained

In [1]:
# %% [markdown]
# # CIFAR-100 Original vs Retrained vs Redistributed Accuracy Table

# %%
import os
import numpy as np
import pandas as pd

from lib.utils import model_results_from_npz  # Your existing function

# %%
# ----- Config -----
NUM_CLASSES = 100
SEED = 969
EPOCHS = 50
CLASSES = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 
    'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 
    'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 
    'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 
    'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 
    'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 
    'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 
    'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 
    'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 
    'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 
    'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm'
]

REMOVE_CLASSES = [
    4, 7, 6, 11, 14, 15, 19, 24, 23, 25, 29, 30, 
    31, 36, 35, 39, 44, 41, 49, 48, 51, 53, 59, 57, 60, 61, 
    68, 67, 72, 71, 76, 77, 80, 84, 88, 85, 92, 94, 99, 97
]  # CIFAR-100 subset

DIR_ORIG   = "../results/cifar100/"
DIR_RETRAIN = "../results/cifar100/"
DIR_REDIS   = "../results/redis_cifar100/"

# %%
# ----- Load original (unlearned) model -----
orig_path = os.path.join(DIR_ORIG, f"cifar100_resnet18_s{SEED}_e{EPOCHS}.npz")
if not os.path.exists(orig_path):
    orig_path = os.path.join(DIR_ORIG, f"cifar100_resnet18_s{SEED}_e{EPOCHS}_r-1.npz")

if not os.path.exists(orig_path):
    raise FileNotFoundError("Original model file not found. Please check filename convention.")

model_orig = model_results_from_npz(orig_path, NUM_CLASSES)

# %%
# ----- Collect results per removed class -----
rows = []

for removed_idx in REMOVE_CLASSES:
    removed_name = CLASSES[removed_idx]

    retrain_path = os.path.join(DIR_RETRAIN, f"cifar100_resnet18_s{SEED}_e{EPOCHS}_r{removed_idx}.npz")
    redis_path   = os.path.join(DIR_REDIS,   f"cifar_resnet_s{SEED}_e{EPOCHS}_rd{removed_idx}.npz")

    if not (os.path.exists(retrain_path) and os.path.exists(redis_path)):
        print(f"Skipping missing class {removed_name}")
        continue

    model_retrain = model_results_from_npz(retrain_path, NUM_CLASSES)
    model_redis   = model_results_from_npz(redis_path, NUM_CLASSES)

    # Helper to compute forget/retain accuracy
    def get_forget_retain(acc, removed_idx):
        forget = acc[removed_idx]
        retain = np.mean([acc[i] for i in range(NUM_CLASSES) if i != removed_idx])
        return forget, retain

    forget_orig, retain_orig = get_forget_retain(model_orig.accuracy, removed_idx)
    forget_retrain, retain_retrain = get_forget_retain(model_retrain.accuracy, removed_idx)
    forget_redis, retain_redis = get_forget_retain(model_redis.accuracy, removed_idx)

    rows.append({
        "removed_class": removed_idx,
        "removed_name": removed_name,

        "orig_retain": retain_orig,
        "orig_forget": forget_orig,

        "retrain_retain": retain_retrain,
        "retrain_forget": forget_retrain,

        "redis_retain": retain_redis,
        "redis_forget": forget_redis,
    })

# %%
# ----- Build DataFrame -----
df = pd.DataFrame(rows)
df = df[[
    "removed_class", "removed_name",
    "orig_retain", "orig_forget",
    "retrain_retain", "retrain_forget",
    "redis_retain", "redis_forget"
]]

df

# Optionally save as CSV:
os.makedirs("../analytics/CIFAR100/accuracy", exist_ok=True)
df.to_csv(f"../analytics/CIFAR100/accuracy/redis_vs_retrain_vs_orig_accuracy_e{EPOCHS}_s{SEED}.csv", index=False)
print(f"Saved CIFAR-100 results for seed {SEED} ✅")


Saved CIFAR-100 results for seed 969 ✅
